## Linear Regression reduction time


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import os
import time

# checkpoint -> result 불러오기
import argparse
from exp.exp_long_term_forecasting import Exp_Long_Term_Forecast
from exp.exp_imputation import Exp_Imputation
from exp.exp_short_term_forecasting import Exp_Short_Term_Forecast
from exp.exp_long_term_forecasting_partial import Exp_Long_Term_Forecast_Partial
from exp.exp_anomaly_detection import Exp_Anomaly_Detection
from exp.exp_classification import Exp_Classification
from utils.print_args import print_args
import random
from utils.metrics import *
from utils.tools import EarlyStopping

from utils.metrics import *
from utils.tools import linear_regression_direct, linear_predict
from data_provider.data_factory import data_provider
from data_provider.data_loader import Dataset_Custom

# 모델 훈련셋 결과 확인하기
from torch.utils.data import DataLoader

# 파서 불러오기
from commons.parser_write import *

# fix random seed
fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

In [9]:
scripts_new = """--task_name long_term_forecast \
  --is_training 1 \
  --root_path ./dataset/weather/ \
  --data_path weather.csv \
  --model_id weather_96_96 \
  --model iTransformer \
  --data custom \
  --features M \
  --seq_len 96 \
  --label_len 48 \
  --pred_len 96 \
  --e_layers 3 \
  --d_layers 1 \
  --factor 3 \
  --enc_in 21 \
  --dec_in 21 \
  --c_out 21 \
  --des 'Exp' \
  --d_model 512\
  --d_ff 512\
  --itr 1 """

In [10]:
args_list2 = []
arg = parser.parse_args(scripts_new.split())
arg.use_gpu = True if torch.cuda.is_available() and arg.use_gpu else False

if arg.use_gpu and arg.use_multi_gpu:
    arg.devices = arg.devices.replace(' ', '')
    device_ids = arg.devices.split(',')
    arg.device_ids = [int(id_) for id_ in device_ids]
    arg.gpu = arg.device_ids[0]
args_list2.append(arg)

In [11]:
weather_96_96_new = "long_term_forecast_weather_96_96_Mod-iTransformer_data-weather.csv_(96to96)_0(1728617672)"


In [12]:
setting_pairs = [
    (weather_96_96_new, args_list2[0])
]

idx = 0 # 순서
col_count = 6 # 한 에포크당 수집 데이터 수
num_epochs = 5 # 에포크 ㅅ횟수
use_gpu = 0 # 사용 GPU 번호 - 오류 잡기 위해 
# q1, q2 = "lin96", "lin24" # 앙상블 모델 텍스트
q1, q2 = "lin96", "lin24" # 앙상블 모델 텍스트
a_init , b_init = 5, -5  # 초기값(sigmoid로변환할  것 감안)  
lr = 0.1 #gradient descending 속도. 0.001이 너무 커서 조정햇습니다.

In [13]:
setting_path = setting_pairs[idx][0]
args = setting_pairs[idx][1]
args.gpu = use_gpu

# 모델 호출 - Exp_Long_Term_Forecast - exchange_96_96
exp_model = Exp_Long_Term_Forecast(args)
exp_model._build_model()
device = torch.device(f"cuda:{use_gpu}")
exp_model.model.device = device
# device = exp_model.device

# 위의 argument와 맞는 모델 호출
checkpoint_path = './checkpoints/'
model_path = f"{checkpoint_path}{setting_path}/checkpoint.pth"
model = torch.load(model_path, map_location="cuda:0")  # 0번 GPU로 매핑
exp_model.model.load_state_dict(model, strict=False)

Use GPU: cuda:0


<All keys matched successfully>

In [14]:
# data_provider -> Exchange_rate
dataset_input = Dataset_Custom(args, args.root_path,
                                    flag='train', size=(args.seq_len, args.label_len, args.pred_len),
                                    features='M', data_path = args.data_path,
                                    target='OT', scale=True, freq='h', timeenc=0,
                                    seasonal_patterns=None, train_ratio=args.train_ratio, test_ratio=args.test_ratio)
dataset_input_test = Dataset_Custom(args, args.root_path,
                                    flag='test', size=(args.seq_len, args.label_len, args.pred_len),
                                    features='M', data_path = args.data_path,
                                    target='OT', scale=True, freq='h', timeenc=0,
                                    seasonal_patterns=None, train_ratio=args.train_ratio, test_ratio=args.test_ratio)

exp_model.model.eval()



dataset_input_loader = DataLoader(
            dataset_input,
            batch_size=args.batch_size,
            shuffle=True,
            num_workers=args.num_workers,
            drop_last=False)
dataset_input_test_loader = DataLoader(
            dataset_input_test,
            batch_size=1, # 모든 데이터셋을 확인해야 해서 batch_size를 강제로 1로 조정.
            shuffle=False,
            num_workers=args.num_workers,
            drop_last=False)

In [15]:
X = np.array([[t] for t in range(-args.seq_len, 0)])  # X는 입력 feature, shape: [seq_len, 1]
X_new = np.array([[t] for t in range(args.pred_len)])  # 예측을 위한 새로운 시간 변수
X_concat = np.concatenate([X, X_new], axis=0).reshape(-1)

In [16]:
# Combination 모델 제작, 2단계/3단계 대응
class CombinedModel(nn.Module):
    # 모델 정의 - 
    def __init__(self, res_A, res_B, res_C):
        super(CombinedModel, self).__init__()
        self.res_A = res_A  # iTransformer train_result
        self.res_B = res_B  # lin_reg_96 train_result
        self.res_C = res_C
        self.a = nn.Parameter(torch.ones(1, device=device)*a_init, requires_grad=True)
        self.b = nn.Parameter(torch.ones(1, device=device)*b_init, requires_grad=True)
        if res_C is not None:
            self.c = nn.Parameter(torch.ones(1, device=device)*(1-a_init -b_init), requires_grad=True)
        else:
            self.c = nn.Parameter(torch.ones(1, device=device)*0.0, requires_grad=True)
        # self.c = nn.Parameter(torch.ones(1, device=device)*0.0, requires_grad=True)
        # self.d = nn.Parameter(torch.zeros(1, device=device), requires_grad=True)
        self.a_sigmoid = torch.sigmoid(self.a)
        self.b_sigmoid = torch.sigmoid(self.b)
    
    def set_a(self, val):
        # nn.Parameter를 다시 생성하지 않고 값을 설정
        with torch.no_grad():
            self.a.copy_(torch.tensor([val], device=device))

    def set_b(self, val):
        # nn.Parameter를 다시 생성하지 않고 값을 설정
        with torch.no_grad():
            self.b.copy_(torch.tensor([val], device=device))
            
    def forward(self, x):
        output_A = self.res_A(x)
        output_B = self.res_B(x)
        
        # Apply sigmoid to ensure non-negative coefficients in range (0, 1)
        a_sigmoid = torch.sigmoid(self.a)
        b_sigmoid = torch.sigmoid(self.b)
        
        # Ensure the sum constraint by defining c as the remainder
        c_sigmoid = torch.clamp(1 - a_sigmoid - b_sigmoid, min=0)
    
        # Compute the combined output with updated a, b, c
        combined_output = a_sigmoid * output_A + b_sigmoid * output_B
        
        self.a_sigmoid = a_sigmoid
        self.b_sigmoid = b_sigmoid
        
    
        if self.res_C is not None:
            output_C = self.res_C(x)
            combined_output += c_sigmoid * output_C
            
    
        return combined_output
    
    def get_result(self):
        a, b = self.a_sigmoid.detach().cpu().numpy()[0], self.b_sigmoid.detach().cpu().numpy()[0]
        return a,b
        

In [18]:
def res_lin_reg(batch_x, reg_size=96, device='cuda:0'):
    B, L, N = batch_x.shape  # L is the sequence length
    # Prepare input tensor by selecting the last `reg_size` elements for each sequence
    X = batch_x[:, -reg_size:, :].to(device)  # Shape: [B, reg_size, N]
    
    # Add bias term to X
    X_b = torch.cat([torch.ones((B, reg_size, 1), device=device), X], dim=2)  # Shape: [B, reg_size, N+1]
    
    # Prepare the output tensor
    lin_result = torch.zeros((B, L, N), device=device)
    
    # Batch computation for linear regression coefficients
    for idx in range(B):
        XTX = X_b[idx].transpose(0, 1) @ X_b[idx]  # Shape: [N+1, N+1]
        XTy = X_b[idx].transpose(0, 1) @ batch_x[idx, -reg_size:, :].to(device)  # Shape: [N+1, N]
        
        # Solve for theta_best in batch
        theta_best = torch.linalg.solve(XTX, XTy)  # Shape: [N+1, N]
        
        # Predict over the entire sequence
        X_full = torch.cat([torch.ones((L, 1), device=device), batch_x[idx, :, :].to(device)], dim=1)  # Shape: [L, N+1]
        lin_result[idx] = X_full @ theta_best  # Shape: [L, N]
    
    return lin_result

In [17]:
# model_output_function

def res_iTransformer(batch_x): # S 
    B, L, N = batch_x.shape  # L은 시퀀스 길이(seq_len)
    return exp_model.model(batch_x, None, torch.zeros(B, len(X_new), N), None)

def res_lin_reg(batch_x, reg_size=96):
    B, L, N = batch_x.shape  # L은 시퀀스 길이(seq_len)
    # 각 배치와 변수에 대해 선형 회귀 해를 계산
    vals = [[linear_regression_direct(X[-reg_size:], batch_x.permute(0,2,1)[idx, var , -reg_size:], device) for var in range(N)] for idx in range(B)]
    lin_result = [[linear_predict(X_new, vals[idx][var], device) for var in range(N)] for idx in range(B)]
    # 결과를 3D 텐서로 변환
    lin_result = torch.stack([torch.stack(lin_result[idx], dim=0) for idx in range(B)], dim=0).to(device).permute(0,2,1)
    return lin_result

def zero_model(batch_x): # S 길이
    B, L, N = batch_x.shape  # L은 시퀀스 길이(seq_len)
    return torch.zeros(B, len(X_new), N)

# 함수 도출
def get_res_lin(reg_size):
    def fn(reg_size):
        return res_lin_reg(reg_size)
    
    return fn
    
def res_lin_reg_24(batch_x):
    B, L, N = batch_x.shape  # L은 시퀀스 길이(seq_len)
    # 24 조각에 대해서도 계산
    vals_24 = [[linear_regression_direct(X[-24:], batch_x.permute(0,2,1)[idx, var , -24:], device) for var in range(N)] for idx in range(B)]
    lin_result_24 = [[linear_predict(X_new, vals_24[idx][var], device) for var in range(N)] for idx in range(B)]
    # 결과를 3D 텐서로 변환
    lin_result_24 = torch.stack([torch.stack(lin_result_24[idx], dim=0) for idx in range(B)], dim=0).to(device).permute(0,2,1)
    return lin_result_24


def res_lin_reg_48(batch_x):
    B, L, N = batch_x.shape  # L은 시퀀스 길이(seq_len)
    # 48 조각에 대해서도 계산
    vals_48 = [[linear_regression_direct(X[-48:], batch_x.permute(0,2,1)[idx, var , -48:], device) for var in range(N)] for idx in range(B)]
    lin_result_48 = [[linear_predict(X_new, vals_48[idx][var], device) for var in range(N)] for idx in range(B)]
    # 결과를 3D 텐서로 변환
    lin_result_48 = torch.stack([torch.stack(lin_result_48[idx], dim=0) for idx in range(B)], dim=0).to(device).permute(0,2,1)
    return lin_result_48

def res_lin_reg_12(batch_x):
    B, L, N = batch_x.shape  # L은 시퀀스 길이(seq_len)
    # 12 조각에 대해서도 계산
    vals_12 = [[linear_regression_direct(X[-12:], batch_x.permute(0,2,1)[idx, var , -12:], device) for var in range(N)] for idx in range(B)]
    lin_result_12 = [[linear_predict(X_new, vals_12[idx][var], device) for var in range(N)] for idx in range(B)]
    # 결과를 3D 텐서로 변환
    lin_result_12 = torch.stack([torch.stack(lin_result_12[idx], dim=0) for idx in range(B)], dim=0).to(device).permute(0,2,1)
    return lin_result_12

QMAP_FN = {
  "none": None,
  "lin12" : res_lin_reg_12,
  "lin24" : res_lin_reg_24,
  "lin48" : res_lin_reg_48,
  "lin96" : res_lin_reg,
  "lin" : res_lin_reg
}

In [19]:
QMAP_FN = {
  "none": None,
  "lin12" : res_lin_reg_12,
  "lin24" : res_lin_reg_24,
  "lin48" : res_lin_reg_48,
  "lin96" : res_lin_reg,
  "lin" : res_lin_reg
}

In [20]:
preds_te_tr = [] # 예측값
trues_te_tr = [] # 참값
preds_te_lin = [] # 96_lin

for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(dataset_input_loader):
    batch_x = batch_x.float().to(device)
    batch_y = batch_y.float().to(device)

    B, L, N = batch_x.shape  # L은 시퀀스 길이(seq_len)

    batch_x_mark = batch_x_mark.float().to(device)
    batch_y_mark = batch_y_mark.float().to(device)

    # decoder input
    dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float()
    dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).float().to(device)
    # encoder - decoder

    # use_amp도 사용하지 않음, 
    outputs = exp_model.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
    
    # 각 배치와 변수에 대해 선형 회귀 해를 계산
    lin_result = res_lin_reg(batch_x, 96).to(device)
    
    outputs = outputs[:, -args.pred_len:, :]
    batch_y = batch_y[:, -args.pred_len:, :].to(device)
    outputs = outputs.detach().cpu().numpy()
    batch_y = batch_y.detach().cpu().numpy()

    pred = outputs
    true = batch_y

    preds_te_tr.append(pred)
    trues_te_tr.append(true)
    preds_te_lin.append(lin_result)

    if (i+1)%100==0:
        print(f"step {i+1} completed")

step 100 completed
step 200 completed
step 300 completed
step 400 completed
step 500 completed
step 600 completed
step 700 completed
step 800 completed
step 900 completed
step 1000 completed
step 1100 completed


In [66]:
def res_lin_reg2(batch_x, reg_size=96):
    B, L, N = batch_x.shape  # L은 시퀀스 길이(seq_len)
    # 각 배치와 변수에 대해 선형 회귀 해를 계산
    vals = [[linear_regression_direct(X[-reg_size:], batch_x.permute(0,2,1)[idx, var , -reg_size:], device) for var in range(N)] for idx in range(B)]
    lin_result = [[linear_predict(X_new, vals[idx][var], device) for var in range(N)] for idx in range(B)]
    # 결과를 3D 텐서로 변환
    lin_result = torch.stack([torch.stack(lin_result[idx], dim=0) for idx in range(B)], dim=0).to(device).permute(0,2,1)
    return lin_result

In [67]:
preds_te_tr2 = [] # 예측값
trues_te_tr2 = [] # 참값
preds_te_lin2 = [] # 96_lin

for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(dataset_input_loader):
    batch_x = batch_x.float().to(device)
    batch_y = batch_y.float().to(device)

    B, L, N = batch_x.shape  # L은 시퀀스 길이(seq_len)

    batch_x_mark = batch_x_mark.float().to(device)
    batch_y_mark = batch_y_mark.float().to(device)

    # decoder input
    dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float()
    dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).float().to(device)
    # encoder - decoder

    # use_amp도 사용하지 않음, 
    outputs = exp_model.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
    
    # 각 배치와 변수에 대해 선형 회귀 해를 계산
    lin_result = res_lin_reg2(batch_x, 96).to(device)
    
    outputs = outputs[:, -args.pred_len:, :]
    batch_y = batch_y[:, -args.pred_len:, :].to(device)
    outputs = outputs.detach().cpu().numpy()
    batch_y = batch_y.detach().cpu().numpy()

    pred = outputs
    true = batch_y

    preds_te_tr2.append(pred)
    trues_te_tr2.append(true)
    preds_te_lin2.append(lin_result)

    if (i+1)%100==0:
        print(f"step {i+1} completed")

step 100 completed
step 200 completed
step 300 completed
step 400 completed
step 500 completed
step 600 completed
step 700 completed
step 800 completed
step 900 completed
step 1000 completed
step 1100 completed


In [21]:
preds_te_tr = np.concatenate(preds_te_tr, axis=0)
trues_te_tr = np.concatenate(trues_te_tr, axis=0)
preds_te_lin = np.transpose(torch.concat(preds_te_lin, axis=0).detach().cpu().numpy(), (0,2,1))

In [68]:
preds_te_tr2 = np.concatenate(preds_te_tr2, axis=0)
trues_te_tr2 = np.concatenate(trues_te_tr2, axis=0)
preds_te_lin2 = np.transpose(torch.concat(preds_te_lin2, axis=0).detach().cpu().numpy(), (0,2,1))

In [22]:
input_len = i + 1

In [23]:
# 모델 실험
combine_model_test = CombinedModel(res_iTransformer, res_lin_reg, None )
# combine_model_test.set_a(0.95)
# combine_model_test.set_b(0.025)
# combine_model_test training
combine_model_test.train()
# torch.nn.utils.clip_grad_norm_(combine_model_test.parameters(), max_norm=0.8) # gradient clipping - 크기 제한

criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(combine_model_test.parameters(), lr=lr, weight_decay=1e-4)
optimizer = torch.optim.Adam([combine_model_test.a, combine_model_test.b], lr=lr)

# 검증 데이터셋 결과 확인
def vali(vali_data, vali_loader, criterion):
    total_loss = []
    combine_model_test.eval()
    len_data = (len(vali_data)-1)//5 
    with torch.no_grad():
        for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(vali_loader):
            batch_x = batch_x.float().to(device)
            batch_y = batch_y.float().to(device)
            targets = batch_y[:, -args.pred_len:, :].to(device)
            outputs = combine_model_test(batch_x)
            loss = criterion(outputs, targets)
            total_loss.append(loss)
            
        
    total_loss = [v.item() for v in total_loss]
    total_loss = np.average(total_loss)
    combine_model_test.train()
    return total_loss

# 모델 훈련

loss_points = [] # (a, b)
# input_len = int(np.ceil(len(dataset_input) / args.batch_size) )
input_len_div = int(np.ceil(input_len / (col_count - 1)))

print("INPUT_LEN", input_len, input_len_div)
early_stopping = EarlyStopping(patience=2, verbose=True)
for epoch in range(num_epochs):
    cnt = 0
    train_loss = []
    # exp_model.train()

    
    path = os.path.join(args.checkpoints, setting_path)
    if not os.path.exists(path):
        os.makedirs(path)
    for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(dataset_input_loader):
        cnt += 1
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)
        targets = batch_y[:, -args.pred_len:, :].to(device)
        optimizer.zero_grad()
        outputs = combine_model_test(batch_x)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss.append(loss)
        if (cnt+1) % 50 == 0:
            print(f"{cnt+1}th batch done, loss {loss}")
        if i == input_len -1 or i % input_len_div == 0:
            a, b = torch.sigmoid(combine_model_test.a).detach().cpu().numpy()[0], 1 - torch.sigmoid(combine_model_test.a).detach().cpu().numpy()[0]
            print(f"STEP {i} , {a,b}", "FIX", (a,b) ==combine_model_test.get_result() )
            loss_points.append((a,b))
            vali_loss = vali(dataset_input_test, dataset_input_test_loader, criterion)
            print("vali_loss:", vali_loss)

    print("="*50)
    print(f"Epoch {epoch+1} DONE")
    print()
    train_loss = [v.item() for v in train_loss]
    train_loss = np.average(train_loss)
    vali_loss = vali(dataset_input_test, dataset_input_test_loader, criterion)
    # print("vali_loss:", vali_loss)
    # print()
    print(a,b)
    # early_stopping(vali_loss, combine_model_test, path)
    # if early_stopping.early_stop:
    #    print("Early stopping")
    #    break
    model_path = path + '/' + 'checkpoint_ensenble.pth'
    # torch.save(combine_model_test.state_dict(), model_path)
    # combine_model_test.load_state_dict(torch.load(model_path, map_location=device))

# 훈련 결과 도출
combine_model_test.eval()

INPUT_LEN 1147 230
STEP 0 , (0.9926085, 0.007391512393951416) FIX False
vali_loss: 0.17986487408679752
50th batch done, loss 0.4444684386253357
100th batch done, loss 0.31252753734588623
150th batch done, loss 0.3307621479034424
200th batch done, loss 0.4776759147644043
STEP 230 , (0.8839026, 0.11609739065170288) FIX False
vali_loss: 0.17683225202690547
250th batch done, loss 0.29288575053215027
300th batch done, loss 1.2355951070785522
350th batch done, loss 0.36371880769729614
400th batch done, loss 0.24600687623023987
450th batch done, loss 0.40978124737739563
STEP 460 , (0.926933, 0.07306700944900513) FIX False
vali_loss: 0.17635128493996838
500th batch done, loss 0.35373401641845703
550th batch done, loss 0.2922360301017761
600th batch done, loss 1.1500778198242188
650th batch done, loss 0.44488468766212463
STEP 690 , (0.9084992, 0.09150081872940063) FIX False
vali_loss: 0.1763129110707734
700th batch done, loss 0.37750521302223206
750th batch done, loss 0.4997476041316986
800th b

CombinedModel()

In [61]:
# 선형회귀 계수 결과값으로 계산
def get_np_pred_lin(np_pred, reg_size=96):
    # 이제 계산도 한다
    # 각 배치와 변수에 대해 선형 회귀 해를 계산
    B, L, N = np_pred.shape  # L은 시퀀스 길이(seq_len)
    vals = [[linear_regression_direct(X[-reg_size:], dataset_input_test[idx][0][-reg_size:, var]) for var in range(N)] for idx in range(B)]
    lin_result = [[linear_predict(X_new, vals[idx][var]) for var in range(N)] for idx in range(B)]
    # 결과를 numpy 모듈로 변경
    np_pred_lin = torch.stack([torch.stack(lin_result[idx], dim=0) for idx in range(B)], dim=0).to(device).permute(0,2,1).detach().cpu().numpy()
    return np_pred_lin

In [26]:
# 실제 데이터 셋 호출
result_list = ['pred.npy', 'true.npy']
result_path = './results/'
np_pred = np.load(f"{result_path}{setting_path}/{result_list[0]}")
np_true = np.load(f"{result_path}{setting_path}/{result_list[1]}")


In [33]:
np_pred.shape

(10444, 96, 21)

In [54]:
def get_np_pred_lin(np_pred, dataset_input_test, reg_size=96, device='cuda:0'):
    B, L, N = np_pred.shape  # B: Batch size, L: Sequence length, N: Number of variables

    # Prepare input tensor for the regression by selecting the last `reg_size` elements
    X  = torch.Tensor(np_pred[:, -reg_size:, :]).to(device)
    # X = np_pred[:, -reg_size:, :]  # Shape: [B, reg_size, N]
    
    # Add bias term to X
    X_b = torch.cat([torch.ones((B, reg_size, 1), device=device), X], dim=2)  # Shape: [B, reg_size, N+1]

    # Initialize tensor for storing linear regression coefficients and predictions
    lin_result = torch.zeros((B, L, N), device=device)

    for idx in range(B):
        # Select data from `dataset_input_test` for the regression target (for each variable)
        target = torch.Tensor(dataset_input_test[idx][0][-reg_size:, :]).to(device)  # Shape: [reg_size, N]

        # Compute batch matrix products X^T X and X^T y
        XTX = X_b[idx].transpose(0, 1) @ X_b[idx]  # Shape: [N+1, N+1]
        XTy = X_b[idx].transpose(0, 1) @ target  # Shape: [N+1, N]

        # Solve for linear regression coefficients (theta_best)
        theta_best = torch.linalg.solve(XTX, XTy)  # Shape: [N+1, N]

        # Use the learned coefficients to make predictions for the full sequence
        X_full = torch.cat([torch.ones((L, 1), device=device), torch.Tensor(np_pred[idx, :, :]).to(device)], dim=1)  # Shape: [L, N+1]
        lin_result[idx] = X_full @ theta_best  # Shape: [L, N]

    # Convert the final result back to NumPy
    # np_pred_lin = lin_result.permute(0, 2, 1).detach().cpu().numpy()  # Shape: [B, N, L]
    np_pred_lin = lin_result.detach().cpu().numpy()
    
    return np_pred_lin

In [62]:
np_pred_first = get_np_pred_lin(np_pred, 96)
np_pred_second = np.zeros(np_pred.shape)

In [63]:
np_pred.shape, np_pred_first.shape

((10444, 96, 21), (10444, 96, 21))

In [64]:
print("COMBI", len(loss_points))

loss_points_map = [] # a,b,c, mae, mse, smae, std_ratio, slope_ratio

# loss_points에서 수집한 도트들을 비교 -> 최소 MSE, 최소 MAE 검색, 최소 SMAE 검색
for j, (a,b) in enumerate(loss_points):
    res_temp = a*np_pred + b*np_pred_first + (1-a-b)*np_pred_second
    mse_step = MSE(res_temp, np_true)
    mae_step = MAE(res_temp, np_true)
    smae_step = SMAE(res_temp, np_true)
    # std_step = STD_RATIO(res_temp, np_true)
    # slope_step = SLOPE_RATIO(res_temp, np_true)
    
    loss_points_map.append({"cnt": j, "a":a,"b":b,"MSE":mse_step,"MAE": mae_step,"SMAE": smae_step})
    # loss_points_map.append({"cnt": j, "a":a,"b":b,"MSE":mse_step,"MAE": mae_step,"SMAE": smae_step, "STD_RATIO": std_step, "slope_step": slope_step})

# MSE 기준으로 정렬
main_key = "MSE" 
new_loss_points_map = sorted(loss_points_map, key=lambda x: x[main_key])
        
# 우선 mae 기준으로 할당.
a, b = new_loss_points_map[0]["a"], new_loss_points_map[0]["b"]

# 마지막으로 비교
final_res = a*np_pred + b* np_pred_first + (1-a-b)*np_pred_second
# final_res = a*np_pred + (1-a)*np_pred_lin

# 메트릭 비교하기 (원본 iTransformer)
with open(f'run_ensenble_txt_{setting_path}_{q1}_{q2}.txt', 'w', encoding='utf8') as A:
    wr = "TRAIN_PRED\n"
    wr += f"{MSE(np_pred, np_true), MAE(np_pred, np_true), SMAE(np_pred, np_true), STD_RATIO(np_pred, np_true), SLOPE_RATIO(np_pred, np_true)} \n"
    wr += "TRAIN_ENSEMBLE_PRED\n"
    wr += f"{MSE(final_res, np_true), MAE(final_res, np_true), SMAE(final_res, np_true), STD_RATIO(final_res, np_true), SLOPE_RATIO(final_res, np_true)}\n"
    wr += "LIN_PRED\n"
    wr += "TRAIN_PRED_FIRST\n"
    wr += f"{MSE(np_pred_first, np_true), MAE(np_pred_first, np_true), SMAE(np_pred_first, np_true), STD_RATIO(np_pred_first, np_true), SLOPE_RATIO(np_pred_first, np_true)}\n"
    wr += "TRAIN_PRED_SECOND\n"
    wr += f"{MSE(np_pred_second, np_true), MAE(np_pred_second, np_true), SMAE(np_pred_second, np_true), STD_RATIO(np_pred_second, np_true), SLOPE_RATIO(np_pred_second, np_true)}\n"
    wr += f"loss_combi : {loss_points}\n"
    A.write(wr)

# 메트릭 저장
metric_path = f"./results/{setting_path}/"
metric_ensemble = [MSE(np_pred, np_true), MAE(np_pred, np_true), SMAE(np_pred, np_true), REC_CORR(np_pred, np_true), STD_RATIO(np_pred, np_true), SLOPE_RATIO(np_pred, np_true)]
np.save(metric_path + "metrics_ensemble.npy", metric_ensemble)
np.save(metric_path + "pred_ensemble.npy", final_res)
np.save(metric_path + "coef_col.npy", loss_points)
np.save(metric_path + "coef_metric.npy", loss_points_map)

COMBI 30


In [65]:
loss_points_map

[{'cnt': 0,
  'a': 0.9926085,
  'b': 0.007391512393951416,
  'MSE': 0.17493676662685725,
  'MAE': 0.21573891760815053,
  'SMAE': 0.024052868946994002},
 {'cnt': 1,
  'a': 0.8839026,
  'b': 0.11609739065170288,
  'MSE': 0.17272305603149193,
  'MAE': 0.21731697488324556,
  'SMAE': 0.021546886682394503},
 {'cnt': 2,
  'a': 0.926933,
  'b': 0.07306700944900513,
  'MSE': 0.17220167736684663,
  'MAE': 0.21524877447282556,
  'SMAE': 0.022538860348570274},
 {'cnt': 3,
  'a': 0.9084992,
  'b': 0.09150081872940063,
  'MSE': 0.17220079146008135,
  'MAE': 0.2159326311270188,
  'SMAE': 0.02211390819012683},
 {'cnt': 4,
  'a': 0.88161683,
  'b': 0.11838316917419434,
  'MSE': 0.17280198001567293,
  'MAE': 0.2174721499927771,
  'SMAE': 0.021494192957934574},
 {'cnt': 5,
  'a': 0.88931775,
  'b': 0.11068224906921387,
  'MSE': 0.17255670464275422,
  'MAE': 0.216967118285253,
  'SMAE': 0.02167172120371889},
 {'cnt': 6,
  'a': 0.88795334,
  'b': 0.11204665899276733,
  'MSE': 0.17259588548450094,
  'MAE': 